# API Call for: 1) Philadelphia County Time Series, 2) City Comparisons in one year, 3) Philadelphia Census Tract-level data for one year 

Written by Donna Leong

August 5, 2020

This notebook contains three parts to make querying the ACS/Census API easier. 

Part 1 allows the user to create a time series for variables within Philadelphia County to track changes over time. 

Part 2 allows the user to create a comparison among different cities and the US national count. This is for a specific year. 

Part 3 allows the user to pull down tract-level information for variables in one year. 

In [ ]:
import requests
import json
import pandas as pd
import pprint    # library for cleanly printing Python data structures
pp = pprint.PrettyPrinter()

In [ ]:
#Here, we are pulling in the master list of variables from the ACS 2018 survey 
acs_vars=pd.read_csv("https://raw.githubusercontent.com/donnaleong/census_api/master/acs_api_var.csv")
mastervars=acs_vars.drop(columns=['Required','Attributes','Limit',"Predicate Type","Group"])
dfx=pd.DataFrame([['NAME', 'Name of Place', 'Meta'], ['GEO_ID', 'GEOID', 'Meta']], columns=list(['Name', 'Label', 'Concept']))
mvars=mastervars.append(dfx, ignore_index=True)

In [ ]:
# 3. Select all the variables that you want to query (up to 50 at a time). You can create many sets of variables within this cell
## You might want to consider making a var list for each topic within SOTC

##You always need to add 'NAME' and 'GEO_ID' to your list 

var1=('NAME',
         'GEO_ID',
        "B03002_001E", #total pop (sanity check: should equal B03002_002E+B01001I_001E)
        "B01001I_001E", #total hispanic pop
        "B03002_002E", #total non hispanic pop
        "B01001H_001E", #total nh white pop
         ### #"B03002_003E", #sanity check for nhwhite
       "B03002_004E", #totl nh black pop
       "B03002_005E", #total nh native american pop
       "B03002_006E", #total nh asian pop
        )

##create as many variables as you want and just set the variable list that you want as v1 in the next section

var2=('NAME',
        'GEO_ID',
     
     )

var3=('NAME',
        'GEO_ID',
     
     )

# **************


# Part 1: Philadelphia County Time Series

Setting up query: https://api.census.gov/data/2018/acs/acs1/examples.html

Tables list: https://api.census.gov/data/2018/acs/acs1/groups.html

Variables list: https://api.census.gov/data/2018/acs/acs1/variables.html



API Key: b8b1cf4fb168f39fb72e6d881580ca7bd06de349



We are trying to mimic a query link like this: 

https://api.census.gov/data/2018/acs/acs1?get=B00001_001E&for=county:*&in=state:*&key=YOUR_KEY_GOES_HERE

but we are parsing the link so that we can easily change it for the different geographies and years

## Part 1.1: Select parameters for your specific data request

make sure to check that all the years are comparable over time

In [ ]:
# 1. Select your years & make sure to check that all the years are comparable over time
years = {'2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010'}

In [ ]:
# 2. Select your dataset
dataset = "acs/acs1"  #american community survey - 1 year estimates

In [ ]:
# 3. Make sure to set the variable list that you want to query as v1. 
v1=var1

## Part 1.2 Set up an index for the variables

In [ ]:
## here, we are creating an index that searches the master variable list for ones that match our variable list (v1) and pulls the names of the variables
var_tot=pd.DataFrame(list(zip(v1)),
              columns=['Name'])
var=pd.merge(var_tot, mvars, on=['Name'], how='left')
var['Label']=var['Label'].str.cat(var['Concept'],sep=" FROM TABLE: ")
var=var.drop(['Concept'], axis=1)

## The table that spits out here is an index created for just the variables we are interested in
var

## Part 1.3 Query the Census API using a loop function

In [ ]:
#set the variables into the second half of the query string 
get_vars=var['Name']     

#setting url query parts 
HOST = "https://api.census.gov/data"
predicates = {}
predicates["get"]= ",".join(var['Name'])
#these below are set to philadelphia, pa
predicates["for"] = "county:101"
predicates["in"]="state:42"
predicates["key"] = "b8b1cf4fb168f39fb72e6d881580ca7bd06de349"

#create a loop to pull down data from 2018-2010
df = pd.DataFrame()

for y in list(years):
    base_url = "/".join([HOST, y, dataset])
    response=requests.get(base_url,params=predicates)
    results=response.text
    data = json.loads(results)
    data1=pd.DataFrame(data)
    data1['year']= y 
    df=df.append(data1, ignore_index=False)
    
df

## Part 1.4 Gotta clean all of the data!

In [ ]:
#This will drop all non-observations in rows
df=df.drop(0)

In [ ]:
# This renames all the columns using our index
col = var['Label'].tolist()
col.extend(['state','county','year'])
col_names=col
print(col_names)
df.columns = col_names
df

## Part 1.5 Export dataframe as csv


In [ ]:
df.to_csv(r'phl_timesries_var1.csv')

# **


# **

# Part 2: Creating comparison across cities


The following code allows us to combine all the variables across different tables to create a query for the API website.

We are trying to mimic a query link like this: 

https://api.census.gov/data/2018/acs/acs1?get=B00001_001E&for=place:*&in=state:*&key=YOUR_KEY_GOES_HERE

but we are parsing the link so that we can easily change it for the different states and places (cities). 

FYI: normally, FIPS are [state][county][subcounty], but if we use place, we don't need to enter a county. Therefore, the FIPS are creating is [state][place]

## Part 2.1: Select parameters for your specific data request


In [ ]:
# 1. Select ONLY ONE YEAR 
years = '2018'

In [ ]:
# 2. Select your dataset
dataset = "acs/acs1"  #american community survey - 1 year estimates

In [ ]:
# 3. Select the [state][place] that you want to compare 
#we are creating the part of the query with geolocation (&for=place:*&in=state:*)
#Add more if you want 

locations=["us:1", #USA avg
            "place:04000&in=state:24", #Baltimore, MD
            "place:07000&in=state:25", #Boston, MA
            "place:14000&in=state:17", #Chicago, IL
            "place:16000&in=state:39", #Cleveland, OH
            "place:22000&in=state:26", #Detroit, MI
            "place:50000&in=state:11", #District of Columbia
            "place:35000&in=state:48", #Houston, TX
            "place:61000&in=state:42", #Pittsburgh, PA
            "place:60000&in=state:42", #Philadelphia, PA
            "place:55000&in=state:04" #Phoenix, AZ
                  ]

In [ ]:
# 4. Select the variable set that you want to use 
v1=var1     

## Part 2.2 Check that the variable list is set to the one we want 
See top of the notebook for all the variable lists

In [ ]:
## Don't touch 
## here, we are creating an index that searches the master variable list for ones that match our variable list (v1) and pulls the names of the variables
var_tot=pd.DataFrame(list(zip(v1)),
              columns=['Name'])
var=pd.merge(var_tot, mvars, on=['Name'], how='left')
var['Label']=var['Label'].str.cat(var['Concept'],sep=" FROM TABLE: ")
var=var.drop(['Concept'], axis=1)

## The table that spits out here is an index created for just the variables we are interested in
var

## Part 2.3 It's loop time

In [ ]:
HOST = "https://api.census.gov/data"

get_vars=var['Name']
get= ",".join(get_vars)
key = "b8b1cf4fb168f39fb72e6d881580ca7bd06de349"
base_url = "/".join([HOST, years, dataset])


#create the loop!
df=pd.DataFrame()

for l in locations:
    full_url="{}?get={}&for={}&key={}".format(base_url,get,l,key)
    response=requests.get(full_url)
    results=response.text
    data=json.loads(results)
    data1=pd.DataFrame(data)
    df=df.append(data1, ignore_index=False)
    
df

## Part 2.4 Clean the data

In [ ]:
# This drops all the non-value rows
df=df.drop(0)

In [ ]:
# This renames all the columns using our index
col = var['Label'].tolist()
col.extend(['state','place'])
col_names=col
df.columns = col_names
df

## Part 2.5 Save as CSV

In [ ]:
#Rename the csv to something that makes sense
df.to_csv(r'citycompare_var1_2018.csv')

# **

# Part 3: Looking at census tract-level data for one year 

# Part 3.1: Select the parameters of your query

In [ ]:
# 1. Select the year you want to use
year = "2018"

In [ ]:
# 2. Select the Census product you want to use (FYI tract-level data not available for 1 year estimates)
dataset = "acs/acs5" 

In [ ]:
# 3. Select the variable set that you want to use (list is at the top of notebook)
v1=var1     

# Part 3.2 Create an index for the variable list 

In [ ]:
## Don't touch 
## here, we are creating an index that searches the master variable list for ones that match our variable list (v1) and pulls the names of the variables
var_tot=pd.DataFrame(list(zip(v1)),
              columns=['Name'])
var=pd.merge(var_tot, mvars, on=['Name'], how='left')
var['Label']=var['Label'].str.cat(var['Concept'],sep=" FROM TABLE: ")
var=var.drop(['Concept'], axis=1)

## The table that spits out here is an index created for just the variables we are interested in
var

# Part 3.3 Call the API 

In [ ]:
HOST = "https://api.census.gov/data"
base_url = "/".join([HOST, year, dataset])

predicates = {}
predicates["get"]= ",".join(var['Name'])
#these are set to all census tracts in philadelphia, pa
predicates["for"] = "tract:*"
predicates["in"]="state:42+county:101"  
predicates["key"] = "b8b1cf4fb168f39fb72e6d881580ca7bd06de349"

#pull down all tract info for certain year
df = pd.DataFrame()

response=requests.get(base_url,params=predicates)
results=response.text
data = json.loads(results)
data1=pd.DataFrame(data)
df=df.append(data1, ignore_index=False)

df

# Part 3.4 Clean the data 

In [1]:
# This drops all the non-value rows
df=df.drop(0)

NameError: name 'df' is not defined

In [ ]:
# This renames all the columns using our index
col = var['Label'].tolist()
col.extend(['state','county', 'tract'])
col_names=col
df.columns = col_names
df

# Part 3.5 Save as CSV

In [ ]:
df.to_csv(r'tract_var1_2018.csv')